# GPLearnFinance3D 功能展示

必要说明:
1. add_ts_function.py是一个自定义的算子库，在这个里面我们可以自由地定义自己想要的算子。其中我也给大家提供了一些可用的基本算子，这些算子我都检查过，是可以使用的.但目前的架构中，需要注意函数的名字不能存在包含关系，原因很简单，我写的是replace，如果有名称的包含关系很可能就出现错误了。以后有时间再改，先写到这个程度能用就行了hhh。
2. 平时使用的时候，有两种方案：1.（个人推荐）直接在这个文件夹GPLearnFinance3D里面另起一个文件直接用。2.把它放到你需要运行的Python的Site-Packages里面，然后当你的算子确定之后就把add_ts_function.py替换进去。就能直接import了


In [1]:
### 导入相关内容
import warnings
import os
import  numpy as np
import pandas as pd
from add_ts_function import ts_std_10, ts_max_10,  ts_mean_10
import genetic
np.random.seed(10)
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', True)
pd.set_option('display.unicode.ambiguous_as_wide', True)
warnings.filterwarnings('ignore')


In [2]:
# 常量定义
stock_path = "D:\\FinanceStock\\FinanceDatabase_xbx\\stock-trading-data-pro\\"
all_stocks = os.listdir(stock_path)
numbers = 0
total_df = pd.DataFrame()
begin_year = pd.Timestamp('2017-01-01')
stop_year = pd.Timestamp('2019-01-01')

In [3]:
# 加载所有股票，这里为了演示就只加载了20只
for stock in all_stocks:
    if stock[:2]!='bj' and stock[:4]!='sh68' and stock[:4]!='sz30' and numbers<=20:
        numbers+=1
        cur_df = pd.read_csv(stock_path+stock,encoding='gbk',skiprows=[0],parse_dates=["交易日期"])
        cur_df = cur_df[(cur_df["交易日期"]<=stop_year)&(cur_df["交易日期"]>=begin_year)]
        cur_df["收益率"] = cur_df["收盘价"].shift(-1) / cur_df["收盘价"] - 1
        cur_df.dropna(subset=["收益率"],inplace=True)
        total_df = pd.concat([total_df,cur_df],ignore_index=True)
total_df.reset_index(inplace=True,drop=True)
total_df = total_df[["股票代码","交易日期","开盘价","最高价","最低价","收盘价","成交量","成交额","收益率"]]
print(len(total_df["股票代码"].unique()))

18


In [4]:
#二位转三维的函数
def make_XY(df, index_name, columns_name, Y_column,):
    '''

    Args:
        df: 输入的dataFrame shape:[trade_dates * stocks,features] [stock000001(trade_dates,features),stock000002(trade_dates,features)...]
        index_name: 交易日期
        columns_name: 股票名称
        Y_column1: 预测的对象

    Returns: X,Y,feature_names

    '''
    df = df.pivot_table(index=[index_name], columns=[columns_name], sort=True, dropna=False)
    Y1 = df.loc[:,(Y_column,)].to_numpy(dtype=np.double)

    df = df.drop([Y_column,],axis=1)
    X_0_len = len(df.index)

    X_1_len = len(df.columns.levels[0]) - 1
    X_2_len = len(df.columns.levels[1])
    return df.to_numpy(dtype=np.double).reshape((X_0_len, X_1_len, X_2_len)), Y1, df.columns.levels[0].drop([Y_column,])


In [5]:
X,Y1, X_feature_names = make_XY(total_df, "交易日期","股票代码", "收益率",)
function_set_sample = ['common_add', 'common_sub', 'common_mul', 'common_div',
                       'common_log', 'common_sqrt', 'common_abs', 'common_inv', 'common_max', 'common_min', 'common_tan',] #'std_10'
my_function = [ts_std_10, ts_max_10,  ts_mean_10,]
function_set = function_set_sample + my_function

In [6]:
print(total_df.shape)
print(X.shape)
print(Y1.shape)

(8132, 9)
(486, 6, 18)
(486, 18)


In [7]:
# 这里的metric的填写是基于fitness.py 文件中的map里面的key值 时间问题这里就展示两个generation
gp_sample = genetic.SymbolicTransformer(generations=2,
                                        population_size=200,
                                        tournament_size=10,
                                        init_depth=(1, 3),
                                        hall_of_fame=100,
                                        n_components=10,
                                        function_set=function_set,
                                        metric="pearson_3d",
                                        const_range=(-1, 1),
                                        p_crossover=0.4,
                                        p_hoist_mutation=0.001,
                                        p_subtree_mutation=0.01,
                                        p_point_mutation=0.01,
                                        p_point_replace=0.4,
                                        parsimony_coefficient="auto",
                                        feature_names=X_feature_names,
                                        max_samples=1, verbose=1,
                                        random_state=0, n_jobs=-2)

In [8]:
sample_weight = np.array([1]*len(X))
gp_sample.fit_3D(X, Y1,feature_names=X_feature_names,sample_weight=sample_weight,need_parallel=True)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     4.55        0.0147213        9         0.071606              N/A     21.67s
   1     4.90        0.0264698        9         0.071606              N/A      0.00s


SymbolicTransformer(const_range=(-1, 1),
                    feature_names=Index(['开盘价', '成交量', '成交额', '收盘价', '最低价', '最高价'], dtype='object'),
                    function_set=['common_add', 'common_sub', 'common_mul',
                                  'common_div', 'common_log', 'common_sqrt',
                                  'common_abs', 'common_inv', 'common_max',
                                  'common_min', 'common_tan',
                                  <functions._Function object at 0x0000029B40D07CD0>,
                                  <functions._Function object at 0x0000029B40D18990>,
                                  <functions._Function object at 0x0000029B40D07C90>],
                    generations=2, init_depth=(1, 3), max_samples=1,
                    metric='pearson_3d', n_jobs=-2, p_crossover=0.4,
                    p_hoist_mutation=0.001, p_point_replace=0.4,
                    parsimony_coefficient='auto', population_size=200,
                    random_state=0, tournament_size=10, verbose=1)

In [9]:
result = gp_sample.show_program(X,Y1,X_feature_names,baseIC=False)
result

,表达式,fitness,OOB fitness,训练集IC,样本外IC,训练集IR,样本外IR,训练集RankIC,样本外RankIC
0,"common_div(common_sqrt(common_mul(收盘价, 成交量)), ...",0.071606,0.0,-0.071606,0.0,0.208007,0.0,0.121365,0.0
1,"common_div(最低价, 收盘价)",0.058264,0.0,0.058264,0.0,0.165544,0.0,-0.143778,0.0
2,"ts_mean_10(common_log(common_min(0.717, 开盘价)))",0.030272,0.0,-0.030272,0.0,0.092371,0.0,0.000000,0.0
3,"common_max(common_sub(common_abs(0.660), ts_ma...",0.027626,0.0,-0.027626,0.0,0.076699,0.0,-0.145136,0.0
4,ts_std_10(ts_std_10(成交额)),0.026459,0.0,-0.026459,0.0,0.076633,0.0,0.000000,0.0
5,"common_add(common_add(common_mul(成交量, 收盘价), co...",0.026243,0.0,-0.026243,0.0,0.070766,0.0,-0.038158,0.0
6,"common_add(common_abs(common_mul(成交额, 成交量)), c...",0.025854,0.0,-0.025854,0.0,0.070298,0.0,-0.023033,0.0
7,"common_div(common_abs(common_mul(成交额, 成交量)), c...",0.025654,0.0,-0.025654,0.0,0.069254,0.0,-0.005245,0.0
8,"common_sub(最低价, 收盘价)",0.025475,0.0,0.025475,0.0,0.073628,0.0,-0.053770,0.0
9,"common_inv(ts_max_10(common_div(收盘价, 成交额)))",0.025246,0.0,-0.025246,0.0,0.078344,0.0,0.000000,0.0


In [10]:
# 在这个例子里面没有提及如何设置验证集所以看不到OOB 和其他样本外指标这是正常的。fitness 就是训练集IC这是由metric决定的，但训练集IC能看到IC的符号
result.to_csv("./presentation_result_best_program.csv")

In [11]:
# 我还开发携带一个追踪机制，把所有代中IC达标(0.02)的因子表达式都放出来，同时记录是在第几代得到的
gp_sample.fit_3D(X, Y1,X_feature_names,baseIC=0.02)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     4.55        0.0147213        9         0.071606              N/A     17.98s
   1     4.90        0.0264698        9         0.071606              N/A      0.00s


SymbolicTransformer(const_range=(-1, 1),
                    feature_names=Index(['开盘价', '成交量', '成交额', '收盘价', '最低价', '最高价'], dtype='object'),
                    function_set=['common_add', 'common_sub', 'common_mul',
                                  'common_div', 'common_log', 'common_sqrt',
                                  'common_abs', 'common_inv', 'common_max',
                                  'common_min', 'common_tan',
                                  <functions._Function object at 0x0000029B40D07CD0>,
                                  <functions._Function object at 0x0000029B40D18990>,
                                  <functions._Function object at 0x0000029B40D07C90>],
                    generations=2, init_depth=(1, 3), max_samples=1,
                    metric='pearson_3d', n_jobs=-2, p_crossover=0.4,
                    p_hoist_mutation=0.001, p_point_replace=0.4,
                    parsimony_coefficient='auto', population_size=200,
                    random_state=0, tournament_size=10, verbose=1)

In [12]:
baseIC_total_result = gp_sample.show_program(X,Y1,X_feature_names,sample_weight=sample_weight,baseIC=True)
baseIC_total_result.to_csv("./presentation_baseIC_total_result.csv")
print(baseIC_total_result)

                                                  表达式   fitness  OOB fitness  \
0                           ts_std_10(ts_std_10(收盘价))  0.022022          0.0   
1      ts_mean_10(common_log(common_min(0.717, 开盘价)))  0.030272          0.0   
2   common_mul(common_log(成交额), common_sub(-0.095,...  0.024732          0.0   
3                   common_sqrt(common_sub(成交量, 收盘价))  0.024528          0.0   
4   common_mul(收盘价, common_div(-0.240, common_inv(...  0.021650          0.0   
..                                                ...       ...          ...   
74            common_add(成交额, common_min(0.717, 开盘价))  0.025288          0.0   
75                    ts_std_10(common_mul(开盘价, 最低价))  0.024435          0.0   
76              common_add(common_mul(成交量, 收盘价), 成交量)  0.026218          0.0   
77  common_min(common_sub(收盘价, 成交额), common_sqrt(最...  0.025288          0.0   
78                   common_abs(common_sub(收盘价, 成交额))  0.025288          0.0   

       训练集IC  样本外IC     训练集IR  样本外IR  训

In [13]:
baseIC_best_result = gp_sample.show_program(X,Y1,X_feature_names,baseIC=False)
baseIC_best_result.to_csv("./presentation_baseIC_best_result.csv")
print(baseIC_best_result)

                                                 表达式   fitness  OOB fitness  \
0  common_div(common_sqrt(common_mul(收盘价, 成交量)), ...  0.071606          0.0   
1                               common_div(最低价, 收盘价)  0.058264          0.0   
2     ts_mean_10(common_log(common_min(0.717, 开盘价)))  0.030272          0.0   
3  common_max(common_sub(common_abs(0.660), ts_ma...  0.027626          0.0   
4                          ts_std_10(ts_std_10(成交额))  0.026459          0.0   
5  common_add(common_add(common_mul(成交量, 收盘价), co...  0.026243          0.0   
6  common_add(common_abs(common_mul(成交额, 成交量)), c...  0.025854          0.0   
7  common_div(common_abs(common_mul(成交额, 成交量)), c...  0.025654          0.0   
8                               common_sub(最低价, 收盘价)  0.025475          0.0   
9        common_inv(ts_max_10(common_div(收盘价, 成交额)))  0.025246          0.0   

      训练集IC  样本外IC     训练集IR  样本外IR  训练集RankIC  样本外RankIC  
0 -0.071606    0.0  0.208007    0.0   0.121365        0.0  
1  0.05826

### 通过手撸因子，校验一下结果

In [15]:
# 挑选一个因子进行复现
# ts_std_10(ts_std_10(成交额)) 0.026459036226093906
total_df["Factor1"] =total_df.groupby("股票代码").apply(lambda x:x["成交额"].rolling(10).std()).reset_index()["成交额"]
total_df["factor"] =total_df.groupby("股票代码").apply(lambda x:x["Factor1"].rolling(10).std()).reset_index()["Factor1"]
result = total_df.groupby("交易日期").apply(lambda x : x["factor"].corr(x["收益率"])).mean()
result

-0.026459036226093892

In [14]:
# common_mul(ts_mean_10(最低价), common_log(最高价)) 0.02009494398134743
total_df["tsmean10"] =total_df.groupby("股票代码").apply(lambda x :x["最低价"].rolling(10).mean()).reset_index()["最低价"]
total_df["factor"] =np.multiply(np.log(total_df["最高价"]),total_df["tsmean10"])
result = total_df.groupby("交易日期").apply(lambda x : x["factor"].corr(x["收益率"])).mean()
result

0.020094943981347427